EXPLORATORY DATA ANALYSIS (EDA) FOR PRICE OPTIMIZATION
=======================================================

KEY QUESTIONS WE'RE ANSWERING:
1. What's the relationship between price and quantity? (elasticity)
2. Which products/segments are most profitable?
3. How does seasonality affect demand?
4. How do competitor prices impact our sales?
5. What price ranges maximize profit?

In [155]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
# Set visualization style best practice for EDA scripts.
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)




In [156]:
# Load the data
df = pd.read_csv('lab_equipment_pricing.csv')
df['date'] = pd.to_datetime(df['date'])

print(f"✓ Data loaded: {len(df):,} records")
print(f"✓ Date range: {df['date'].min().date()} to {df['date'].max().date()}")

df.info()
''' We don't have any missing values or any datatype issuse. 
(Other wise we would have to us df['col']=df['col'].astype['string/object/int'])   '''

✓ Data loaded: 10,000 records
✓ Date range: 2021-01-01 to 2023-12-31
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  10000 non-null  datetime64[ns]
 1   product               10000 non-null  object        
 2   customer_segment      10000 non-null  object        
 3   price                 10000 non-null  float64       
 4   competitor_price      10000 non-null  float64       
 5   quantity_sold         10000 non-null  int64         
 6   unit_cost             10000 non-null  float64       
 7   inventory_level       10000 non-null  int64         
 8   days_since_promotion  10000 non-null  int64         
 9   competitor_promotion  10000 non-null  int64         
 10  month                 10000 non-null  int64         
 11  quarter               10000 non-null  int64         
 12  day_of

" We don't have any missing values or any datatype issuse. \n(Other wise we would have to us df['col']=df['col'].astype['string/object/int'])   "

In [157]:
df.head()

,date,product,customer_segment,price,competitor_price,quantity_sold,unit_cost,inventory_level,days_since_promotion,competitor_promotion,month,quarter,day_of_week,revenue,cost,profit,margin_percent
0,2023-11-11,Pipettes,Academic,269.04,250.53,70,114.0,87,116,0,11,4,5,18832.80,7980.0,10852.80,57.63
1,2023-02-09,Microscope,Pharma,15998.99,17402.27,115,5700.0,187,20,0,2,1,3,1839883.85,655500.0,1184383.85,64.37
2,2021-09-10,PCR_System,Pharma,7836.67,7509.45,208,3040.0,169,14,0,9,3,4,1630027.36,632320.0,997707.36,61.21
3,2023-04-12,Microscope,Government,14850.43,15919.49,107,5700.0,72,166,0,4,2,2,1588996.01,609900.0,979096.01,61.62
4,2021-01-14,Centrifuge,Government,13110.23,12597.92,78,4560.0,83,91,0,1,1,3,1022597.94,355680.0,666917.94,65.22


In [158]:
print("\n" + "="*80)
print("ABOUT THE DATA")
print("\n" + "="*80)

'''Object is pandas generic type for anything that doesn't fit neatly into 
numeric categories.'''
print("\nCategorical Features and Unique Values:")
print(df.select_dtypes(include=['object']).nunique().sort_values(ascending=False))
print()

print("Size and shape of the DataFrame:")
print(df.size, df.shape)
print()


ABOUT THE DATA


Categorical Features and Unique Values:
product             5
customer_segment    4
dtype: int64

Size and shape of the DataFrame:
170000 (10000, 17)



In [159]:
# ============================================================================
# SECTION 1: DATA QUALITY CHECK
# ============================================================================
print("\n" + "="*80)
print("SECTION 1: DATA QUALITY CHECK")
print("="*80)
print()

print("Missing Values")
print("-"*80)

missing = df.isnull().sum()
if missing.sum() == 0:
    print("No missing values found")
else:
    print("Missing values detected:")
    print(missing[missing > 0])
print()

print("Duplicate Records")
print("-"*80)
duplicates = df.duplicated().sum()
print(f"Duplicate rows: {duplicates}")
if duplicates > 0:
    print("Duplicates found!")
print()

print("Basic Statistics")
print("-"*80)
print(df.describe())
print()

# BUSINESS INSIGHT: Check for unrealistic values
print("Data Sanity Checks")
print("-"*80)
print(f"✓ Negative prices: {(df['price'] < 0).sum()}")
print(f"✓ Negative quantities: {(df['quantity_sold'] < 0).sum()}")
print(f"✓ Negative profits: {(df['profit'] < 0).sum()} ({(df['profit'] < 0).sum() / len(df) * 100:.2f}%)")
print(f"✓ Negative reveunes: {(df['revenue'] < 0).sum()}")
print()

print("PRICING INSIGHT: Negative profits are normal (lost money on some sales).")
print("PRICING INSIGHT: Zero quantities mean price was too high for that context.")
print()



SECTION 1: DATA QUALITY CHECK

Missing Values
--------------------------------------------------------------------------------
No missing values found

Duplicate Records
--------------------------------------------------------------------------------
Duplicate rows: 0

Basic Statistics
--------------------------------------------------------------------------------
                             date         price  competitor_price  \
count                       10000  10000.000000      10000.000000   
mean   2022-06-28 05:29:19.680000   7081.918057       7084.946605   
min           2021-01-01 00:00:00    255.030000        230.430000   
25%           2021-09-30 00:00:00    413.860000        410.767500   
50%           2022-06-28 00:00:00   7969.815000       7937.675000   
75%           2023-03-25 00:00:00  12765.317500      12530.507500   
max           2023-12-31 00:00:00  17243.740000      18879.000000   
std                           NaN   5991.305862       6018.017541   

       qu

### Next let's calculate Elasticity!
WHY? Elasticity tells us how demand responds to price changes.

Formula: % Change in Quantity / % Change in Price

Elastic (< -1): Customers very price-sensitive, lower price = more revenue
Inelastic (> -1): Customers less price-sensitive, can raise prices

Rule of thumb:
- Elastic: |E|>1 (quantity changes proportionally more than price)
- Unit elastic: |E|=1
- Inelastic: |E|<1 (quantity changes proportionally less than price).



In [160]:

# ============================================================================
# SECTION 2: PRICE ELASTICITY ANALYSIS
# ============================================================================
print("\n" + "="*80)
print("SECTION 2: PRICE ELASTICITY ANALYSIS")
print("="*80)

print("Price Elasticity by Product")
print("-"*80)

elasticity_results = []

for product in df['product'].unique():
    product_data = df[df['product'] == product].copy()
    
    # Create price bins (quintiles)
    product_data['price_quintile'] = pd.qcut(product_data['price'], q=5, 
                                              labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'],
                                              duplicates='drop')
    
    # Calculate average quantity by price level
    # observed=True means only include groups that actually exist in the data
    price_qty = product_data.groupby('price_quintile', observed=True).agg({
        'price': 'mean',
        'quantity_sold': 'mean'
    }).sort_values('price')

    
    
    # Calculate elasticity (comparing highest vs lowest quintile)
    if len(price_qty) >= 2:
        price_low = price_qty.iloc[0]['price']
        price_high = price_qty.iloc[-1]['price']
        qty_low = price_qty.iloc[0]['quantity_sold']
        qty_high = price_qty.iloc[-1]['quantity_sold']
        
        # Percent changes
        price_change_pct = (price_high - price_low) / price_low
        qty_change_pct = (qty_high - qty_low) / qty_low
        
        # Elasticity
        if price_change_pct != 0:
            elasticity = qty_change_pct / price_change_pct
            
            elasticity_results.append({
                'Product': product,
                'Elasticity': elasticity,
                'Interpretation': 'Elastic' if elasticity < -1 else 'Inelastic',
                'Avg_Price': product_data['price'].mean(),
                'Price_StdDev': product_data['price'].std()
            })
            
            print(f"{product:15} → Elasticity: {elasticity:6.2f} ({elasticity_results[-1]['Interpretation']:10})")


print()
print("PRICING INSIGHT:")
elasticity_df = pd.DataFrame(elasticity_results).sort_values('Elasticity')
most_elastic = elasticity_df.iloc[0]
least_elastic = elasticity_df.iloc[-1]
print(f"  Most elastic (price-sensitive): {most_elastic['Product']} ({most_elastic['Elasticity']:.2f})")
print(f"  → Strategy: Compete on price, focus on volume")
print(f"  Least elastic (price-insensitive): {least_elastic['Product']} ({least_elastic['Elasticity']:.2f})")
print(f"  → Strategy: Premium pricing opportunity")
print()


SECTION 2: PRICE ELASTICITY ANALYSIS
Price Elasticity by Product
--------------------------------------------------------------------------------
Pipettes        → Elasticity:  -1.39 (Elastic   )
Microscope      → Elasticity:  -0.57 (Inelastic )
PCR_System      → Elasticity:  -0.89 (Inelastic )
Centrifuge      → Elasticity:  -0.67 (Inelastic )
Reagent_Kit     → Elasticity:  -1.13 (Elastic   )

PRICING INSIGHT:
  Most elastic (price-sensitive): Pipettes (-1.39)
  → Strategy: Compete on price, focus on volume
  Least elastic (price-insensitive): Microscope (-0.57)
  → Strategy: Premium pricing opportunity



In [161]:
print("Basic Correlation Analysis")

correlations = df[['price', 'quantity_sold', 'profit', 'competitor_price', 
                    'inventory_level', 'days_since_promotion']].corr()['quantity_sold'].sort_values()

print("Correlation with Quantity Sold:")
print(correlations)
print()

print("PRICING INSIGHT:")
if correlations['price'] < 0:
    print(f"  ✓ Price is negatively correlated with quantity ({correlations['price']:.3f})")
    print(f"    → Normal demand curve behavior confirmed")
else:
    print(f"  ⚠ Unexpected positive correlation - investigate")
print()

Basic Correlation Analysis
Correlation with Quantity Sold:
competitor_price       -0.033201
price                  -0.032566
days_since_promotion   -0.003659
inventory_level         0.014449
profit                  0.316287
quantity_sold           1.000000
Name: quantity_sold, dtype: float64

PRICING INSIGHT:
  ✓ Price is negatively correlated with quantity (-0.033)
    → Normal demand curve behavior confirmed



### PROFITABILITY ANALYSIS

WHY: Identify which products/segments drive profit (focus optimization there)

In [162]:

# ============================================================================
# SECTION 3: PROFITABILITY ANALYSIS
# ============================================================================
print("\n" + "="*80)
print("SECTION 3: PROFITABILITY ANALYSIS")
print("="*80)


print("Profitability by Product")
print("-"*80)

product_profit = df.groupby('product').agg({
    'revenue': 'sum',
    'cost': 'sum',
    'profit': 'sum',
    'quantity_sold': 'sum',
    'price': 'mean'
}).round(0)

product_profit['profit_margin_%'] = (product_profit['profit'] / product_profit['revenue'] * 100).round(3)
product_profit['profit_per_unit'] = (product_profit['profit'] / product_profit['quantity_sold']).round(2)
product_profit = product_profit.sort_values('profit', ascending=False)

# Convert revenue, cost, profit to millions for readability
product_profit[['revenue', 'cost', 'profit']] = product_profit[['revenue', 'cost', 'profit']] / 1_000_000
product_profit = product_profit.round({'revenue': 2, 'cost': 2, 'profit': 2})

print("Profitability by Product (Revenue, Cost, Profit in Millions)")
print(product_profit)
print()


print("PRICING INSIGHT:")
top_profit_product = product_profit.index[0]
low_profit_product = product_profit.index[-1]
print(f"  Highest profit: {top_profit_product} (${product_profit.loc[top_profit_product, 'profit']:.2f}M)")
print(f"  → Priority product for optimization. High-margin pricing insights suggest capitalizing on value-based strategies, premium positioning, or operational efficiency to maintain high per-unit profits.")
print(f"  Highest profit: {low_profit_product} (${product_profit.loc[low_profit_product, 'profit']:.2f}M)")
print(f"  → Low-margin insights point to using volume-based strategies like cost leadership, tiered pricing, or bundling to drive profitability through higher sales volume")
print()




SECTION 3: PROFITABILITY ANALYSIS
Profitability by Product
--------------------------------------------------------------------------------
Profitability by Product (Revenue, Cost, Profit in Millions)
             revenue     cost   profit  quantity_sold    price  \
product                                                          
Microscope   3127.87  1196.91  1930.96         209834  14976.0   
Centrifuge   2508.91   961.30  1547.62         210649  11988.0   
PCR_System   1736.82   665.58  1071.25         218749   8013.0   
Reagent_Kit    96.50    37.22    59.28         217449    449.0   
Pipettes       64.46    24.92    39.54         218394    300.0   

             profit_margin_%  profit_per_unit  
product                                        
Microscope            61.734          9202.33  
Centrifuge            61.685          7346.90  
PCR_System            61.678          4897.14  
Reagent_Kit           61.426           272.61  
Pipettes              61.343           181.07  

In [163]:


print("Profitability by Customer Segment")
print("-"*80)

segment_profit = df.groupby('customer_segment').agg({
    'revenue': 'sum',
    'profit': 'sum',
    'quantity_sold': 'sum',
    'price': 'mean',
    'customer_segment': 'count'  # Count unique customers per segment
}).round(0)

segment_profit.rename(columns={'customer_segment': 'num_customers'}, inplace=True)
segment_profit['profit_margin_%'] = (segment_profit['profit'] / segment_profit['revenue'] * 100).round(1)
segment_profit['profit_per_customer'] = (segment_profit['profit'] / segment_profit['num_customers']).round(2)
segment_profit = segment_profit.sort_values('profit', ascending=False)
print(segment_profit)
print()

print("PRICING INSIGHT:")
top_segment = segment_profit.index[0]
bottom_segment = segment_profit.index[-1]
print(f"  Highest profit segment: {top_segment}")
print(f"  Profit per customer: ${segment_profit.loc[top_segment, 'profit_per_customer']:,.2f}")
print(f"  Lowest profit segment: {bottom_segment}")
print(f"  Profit per customer: ${segment_profit.loc[bottom_segment, 'profit_per_customer']:,.2f}")
print(f"  → Consider different pricing strategies by segment")
print()


Profitability by Customer Segment
--------------------------------------------------------------------------------
                       revenue        profit  quantity_sold   price  \
customer_segment                                                      
Pharma            2.543981e+09  1.572410e+09         361619  7158.0   
Biotech           2.003843e+09  1.235661e+09         286700  7091.0   
Government        1.793477e+09  1.107301e+09         251228  7173.0   
Academic          1.193273e+09  7.332750e+08         175528  6909.0   

                  num_customers  profit_margin_%  profit_per_customer  
customer_segment                                                       
Pharma                     2459             61.8            639450.97  
Biotech                    2456             61.7            503119.25  
Government                 2541             61.7            435773.80  
Academic                   2544             61.5            288237.03  

PRICING INSIGHT:
  Highes


### SEASONALITY ANALYSIS

WHY: Identify when we can charge premium prices (high demand periods)

In [164]:


# ============================================================================
# SECTION 4: SEASONALITY ANALYSIS
# ============================================================================
print("\n" + "="*80)
print("SECTION 4: SEASONALITY ANALYSIS")
print("="*80)
print()

print("Sales by Month")
print("-"*80)

monthly_sales = df.groupby('month').agg({
    'quantity_sold': 'sum',
    'revenue': 'sum',
    'profit': 'sum',
    'price': 'mean'
}).round(0)

monthly_sales['quantity_index'] = (monthly_sales['quantity_sold'] / monthly_sales['quantity_sold'].mean() * 100).round(0)
monthly_sales = monthly_sales.sort_values('quantity_sold', ascending=False)

print(monthly_sales)
print()

high_months = monthly_sales[monthly_sales['quantity_index'] > 110].index.tolist()
low_months = monthly_sales[monthly_sales['quantity_index'] < 90].index.tolist()

print("PRICING INSIGHT:")
print(f"  High season months: {high_months}")
print(f"  → Opportunity to raise prices (demand is high)")
print(f"  Low season months: {low_months}")
print(f"  → May need competitive pricing or promotions")
print()




SECTION 4: SEASONALITY ANALYSIS

Sales by Month
--------------------------------------------------------------------------------
       quantity_sold      revenue       profit   price  quantity_index
month                                                                 
9             118921  777229382.0  477698980.0  6666.0           133.0
10            118250  820436199.0  506773635.0  7028.0           132.0
1             102287  745840851.0  458559556.0  7273.0           114.0
5             100257  731842778.0  451716303.0  7348.0           112.0
12             91056  638116631.0  393570929.0  7104.0           102.0
11             89154  628244870.0  388812298.0  7033.0           100.0
3              87637  616534883.0  382022139.0  7115.0            98.0
4              86379  613619934.0  380099629.0  7214.0            96.0
2              82245  569738703.0  351397730.0  6915.0            92.0
8              68225  495647051.0  304789618.0  7323.0            76.0
7              671

In [165]:
customer_success_per_quarter=df.groupby([ 'quarter','customer_segment']).agg({
'profit': 'sum',
'revenue': 'sum',
'quantity_sold': 'sum',
}).sort_values(['quarter','profit'],ascending=[1,0]).reset_index()
customer_success_per_quarter['profit']=(customer_success_per_quarter['profit']/1000_000).round(2)
customer_success_per_quarter['revenue']=(customer_success_per_quarter['revenue']/1000_000).round(2)
customer_success_per_quarter['Margin_%']=(customer_success_per_quarter['profit']/customer_success_per_quarter['revenue']*100).round(2)  
customer_success_per_quarter.rename(columns={'profit':'Total_Profit_Millions',
                                             'quantity_sold':'Total_Quantity_Sold',
                                             'revenue':'Total_Revenue_Millions',
                                             'quarter':'Quarter',
                                             'customer_segment':'Customer_Segment'
                                             },inplace=True)
customer_success_per_quarter['Profit_per_Unit']=(customer_success_per_quarter['Total_Profit_Millions']*1_000_000/customer_success_per_quarter['Total_Quantity_Sold']).round(2)
print("PRICING INSIGHT:")
print(customer_success_per_quarter[['Quarter','Customer_Segment','Total_Quantity_Sold','Margin_%']])

PRICING INSIGHT:
    Quarter Customer_Segment  Total_Quantity_Sold  Margin_%
0         1           Pharma                92206     61.90
1         1          Biotech                71472     61.44
2         1       Government                64841     61.87
3         1         Academic                43650     61.36
4         2           Pharma                86201     61.75
5         2          Biotech                69138     62.09
6         2       Government                54221     61.53
7         2         Academic                40607     61.51
8         3           Pharma                86500     61.56
9         3          Biotech                68939     61.56
10        3       Government                59941     61.81
11        3         Academic                38899     61.06
12        4           Pharma                96712     61.98
13        4          Biotech                77151     61.57
14        4       Government                72225     61.73
15        4         Aca

### COMPETITIVE DYNAMICS

In [166]:

# ============================================================================
# SECTION 5: COMPETITIVE DYNAMICS
# ============================================================================
print("\n" + "="*80)
print("SECTION 5: COMPETITIVE DYNAMICS")
print("="*80)
print("\nWHY: Understand how competitor pricing affects our performance")
print()

print("Price Positioning vs Competitors")
print("-"*80)

df['price_difference'] = df['price'] - df['competitor_price']
df['price_position'] = pd.cut(df['price_difference'], 
                               bins=[-np.inf, -100, 100, np.inf],
                               labels=['Below Competitor', 'At Parity', 'Above Competitor'])

position_analysis = df.groupby(['price_position','customer_segment']
                               , observed=True).agg({
    'quantity_sold': 'mean',
    'profit': 'mean',
    'revenue': 'mean'
}).round(0).reset_index()


position_analysis['margin_%']= (position_analysis['profit']/position_analysis['revenue']*100).round(2) 
position_analysis=position_analysis.sort_values(['price_position','customer_segment',
                                                 'margin_%']
                                                   ,ascending=False)
print(position_analysis.drop(columns=['revenue','profit']))
print()

print("PRICING INSIGHT:")

print("Pricing Below Competitor yields a high average margin (61.74%), " \
"which is functionally similar to or even slightly higher than the margin achieved when pricing Above Competitor (61.72%). This implies that the Below Competitor strategy is highly selective—it is only used for high-efficiency deals (likely in the Pharma segment, which hits its highest margin, 62.06%, in this tier")
print("The lowest average margin across all segments is found in the At Parity tier (average margin: 61.09%).")
print()



SECTION 5: COMPETITIVE DYNAMICS

WHY: Understand how competitor pricing affects our performance

Price Positioning vs Competitors
--------------------------------------------------------------------------------
      price_position customer_segment  quantity_sold  margin_%
11  Above Competitor           Pharma          145.0     61.78
10  Above Competitor       Government          100.0     61.77
9   Above Competitor          Biotech          116.0     61.79
8   Above Competitor         Academic           68.0     61.55
7          At Parity           Pharma          149.0     60.77
6          At Parity       Government           99.0     61.11
5          At Parity          Biotech          118.0     61.30
4          At Parity         Academic           70.0     61.19
3   Below Competitor           Pharma          147.0     62.06
2   Below Competitor       Government           99.0     61.85
1   Below Competitor          Biotech          115.0     61.63
0   Below Competitor         Aca

### Competition and Inventory Impact 

In [167]:
print(" Competitor Promotion Impact")
print("-"*80)

promo_impact = df.groupby( ['competitor_promotion']
                          ).agg({
    'quantity_sold': 'mean',
    'profit': 'mean',
    'price': 'mean'
}).round(0)

promo_impact.index = ['No Competitor Promo', 'Competitor Promo Active']
print(promo_impact)
print()

if len(promo_impact) > 1:
    qty_drop = ((promo_impact.loc['Competitor Promo Active', 'quantity_sold'] - 
                 promo_impact.loc['No Competitor Promo', 'quantity_sold']) / 
                promo_impact.loc['No Competitor Promo', 'quantity_sold'] * 100)
    
    print(f"PRICING INSIGHT:")
    print(f"When competitor runs promotion, our quantity doesn't drop, meaning low impact of competitor sales, \n actual increase in quantity sold: {qty_drop:.1f}%")
 
print()



 Competitor Promotion Impact
--------------------------------------------------------------------------------
                         quantity_sold    profit   price
No Competitor Promo              107.0  463820.0  7064.0
Competitor Promo Active          108.0  471067.0  7189.0

PRICING INSIGHT:
When competitor runs promotion, our quantity doesn't drop, meaning low impact of competitor sales, 
 actual increase in quantity sold: 0.9%



In [175]:

print("\n" + "="*80)
print("INVENTORY IMPACT ON PRICING")
print("="*80)
print("\nWHY: High inventory creates pressure to lower prices")
print()

df['inventory_category'] = pd.qcut(df['inventory_level'], q=3, labels=['Low', 'Medium', 'High'])

inventory_impact = df.groupby('inventory_category', observed=True).agg({
    'price': 'mean',
    'quantity_sold': 'mean',
    'profit': 'mean'
}).round(0)

print(inventory_impact)
print()

print("PRICING INSIGHT:")
print("Given the demonstrated price inelasticity across previous data points, the pricing\n" \
"for \"Medium\" inventory could likely be raised to match/exceed the \"High\" inventory pricing, \n" \
"resulting in an uplift in total profit with a low risk of loss in sales volume.")

print()




INVENTORY IMPACT ON PRICING

WHY: High inventory creates pressure to lower prices

                     price  quantity_sold    profit
inventory_category                                 
Low                 7143.0          107.0  465966.0
Medium              6918.0          107.0  455405.0
High                7185.0          109.0  473250.0

PRICING INSIGHT:
Given the demonstrated price inelasticity across previous data points, the pricing
for "Medium" inventory could likely be raised to match/exceed the "High" inventory pricing, 
resulting in an uplift in total profit with a low risk of loss in sales volume.

